# Crypto

## Welcome to your Coinbase Crypto Tracker! 🚀📈

Track your gains across diverse assets and stay ahead in the exciting world of cryptocurrencies.

In [1]:
# -- LIBRARIES
import pandas as pd
import numpy as np
import requests
import os
import json
import pprint
import pprint
import time
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

# Ignoring the Deprecation Warning
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# -- FUNCTIONS
def prepareData():
    
    """Imports and cleans the file downloaded from Coinbase
        
        Output:
            DataFrame containing Coinbase transactions 
        """
    
    # Importing the data from Coinbase
    transactions = pd.read_csv('/data/notebook_files/coinbase_statement', skiprows=3)
    
    # Removing spaces and lowercasing the column names
    transactions.columns = (transactions.columns
                            .str.lower()
                            .str.replace(" ","_"))
    
    # Adding date column and dropping unwanted cols
    transactions.drop(columns=['notes'], inplace=True)
    transactions['timestamp'] = pd.to_datetime(transactions['timestamp'])
    transactions['date'] = transactions['timestamp'].dt.date
    
    # Replacing the ETH2 to ETH
    transactions['asset'] = transactions['asset'].replace('ETH2', 'ETH')

    # Keeping only Staking Income and Buy transaction types
    trans_val = ['Buy']
    transactions = transactions[transactions.transaction_type.isin(trans_val)]

    return(transactions)

def callCryptoPrice(crypto: list, curr: str):

    """Calls the CoinMarketCap API to return the current crypto assets price
        
        Input: 
            crypto = list of all crypto assets that we want the price for
            curr = currency of the assets (EUR, CZK, USD, etc.)
        
        Output:
            DataFrame containing current asset prices
    """

    # Coinmarketcap API URL
    url_crypto = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest' 
    
    # Headers for the API request
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': os.environ['coinmarket_api']
    } 
    # Create new session object to manage API requests
    session = Session() 
    
    # Update the session headers with the specified headers
    session.headers.update(headers) 

    # Creating an empty DataFrame
    coins_info = pd.DataFrame(columns=['coin', 'time_stamp', 'currency',
                                       'price', 'percent_change_24h', 'percent_change_30d'])
    
    # Loop through all coins in the list
    for coin in crypto:
        # API parameters to pass in for retrieving specific cryptocurrency data
        parameters = {'symbol': coin, 'convert': curr} 

        # Receiving the response from the API
        response = session.get(url_crypto, params=parameters) 

        # Extracting the coin ID from the JSON output
        info = json.loads(response.text)
        data_coinmarket = info['data'][coin]['quote'][curr]

        # Defining the values for each coin i.e. the price and timestamp of the last updated price
        time_stamp = data_coinmarket['last_updated']
        currency = curr
        price = data_coinmarket['price']
        percent_change_24h = round(data_coinmarket['percent_change_24h'], 2)
        percent_change_30d = round(data_coinmarket['percent_change_30d'], 2)
        
        # Combining all the values into a list and transforming it into a dataframe
        help_list = [[coin, time_stamp, currency, price, percent_change_24h, percent_change_30d]]
        help_df = pd.DataFrame(help_list, columns=['coin', 'time_stamp', 'currency',
                                                   'price', 'percent_change_24h', 'percent_change_30d'])
        coins_info = pd.concat([help_df, coins_info])
    
    return(coins_info)

def mergingDataFrames():

    """Joins the Coinbase transactions file and the current price of crypto assets
        
        Output:
            DataFrame containing crypto transactions and their current price
    """

    #Setting up the list of all cryptos that we want to request the price for from the API
    joined_data = (prepareData()
                   .merge(callCryptoPrice(('ETH', 'BTC', 'ADA', 'DOT', 'SOL'), 'EUR')
                                  , how='left', left_on='asset', right_on='coin'))
    joined_data.dropna(inplace=True)
    
    return(joined_data)

def dataSummary():
    
    """Calculates all important metric values. Such as profit, total invested amount, etc.
        
        Output:
            total_profit = current profit from the invested amount
            total_profit_per = percentage current profit from the invested amount 
            total_value = invested amount into all crypto assets
    """

    # Dropping unwanted columns
    final_data = mergingDataFrames().drop(columns=['transaction_type','coin','time_stamp'])

    # Creating fraction price variables
    final_data['current_fraction_price'] = final_data['price'] * final_data['quantity_transacted']
    final_data['bought_amount'] = final_data['spot_price_at_transaction'] * final_data['quantity_transacted']

    # Calculating profit
    final_data['profit'] = (final_data['current_fraction_price']
                            - final_data['bought_amount']
                            - (final_data['fees_and/or_spread']/24.62))
    
    # Summary dataframe
    summary = (final_data
            .groupby('asset')
            .agg({'quantity_transacted':'sum', 'bought_amount':'sum', 'profit':'sum'})
            .round(2)
            .reset_index()
            .sort_values(by='profit', ascending=False))
    summary['profit_in_pct'] = round((summary['profit'] / summary['bought_amount']) * 100, 2).astype(str) + ' %'

    # Creating statistics
    total_profit = round(sum(summary['profit']), 2)
    total_bought = round(sum(final_data['bought_amount']), 2)
    total_value = total_bought + total_profit
    total_profit_per = round((total_profit/total_bought) * 100, 2)
        
    return total_profit, total_value, total_profit_per #summary.reset_index().drop(columns=['index'])

def cryptoTable():
    
    crypto_data = mergingDataFrames().drop(columns=['transaction_type','coin','time_stamp'])

    # Creating fraction price variables
    crypto_data['current_fraction_price'] = crypto_data['price'] * crypto_data['quantity_transacted']
    crypto_data['bought_amount'] = crypto_data['spot_price_at_transaction'] * crypto_data['quantity_transacted']

    # Calculating profit
    crypto_data['profit'] = (crypto_data['current_fraction_price']
                            - crypto_data['bought_amount']
                            - (crypto_data['fees_and/or_spread']/24.62))

    # Summary dataframe
    summary = (crypto_data
            .groupby('asset')
            .agg({'quantity_transacted':'sum', 'bought_amount':'sum', 'profit':'sum'})
            .round(2)
            .reset_index()
            .sort_values(by='profit', ascending=False))
    summary['profit_in_pct'] = round((summary['profit'] / summary['bought_amount']) * 100, 2).astype(str) + ' %'

    return summary.reset_index().drop(columns=['index'])

profit, value, profit_per = dataSummary()

In [2]:
a = cryptoTable()
a

,asset,quantity_transacted,bought_amount,profit,profit_in_pct
0,BTC,0.03,975.14,670.70,68.78 %
1,ETH,0.71,1536.29,554.26,36.08 %
2,SOL,1.08,82.02,72.36,88.22 %
3,DOT,25.00,218.16,-56.68,-25.98 %
4,ADA,725.76,665.10,-355.75,-53.49 %


In [3]:
import plotly.graph_objects as go

current_value = a.bought_amount + a.profit

fig = go.Figure(data=[
    go.Bar(name="Invested amount", x=a.asset, y=a.bought_amount),
    go.Bar(name="Current value", x=a.asset, y=current_value)
])

fig.update_layout(barmode='group',
                  title='Invested Amount in Crypto Assets',
                  xaxis=dict(title="Assets"),
                  yaxis=dict(title="Value (in EUR)"))
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()